<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Librarys" data-toc-modified-id="Importing-Librarys-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Librarys</a></span></li><li><span><a href="#Fake-News-Classifier-Using-LSTM" data-toc-modified-id="Fake-News-Classifier-Using-LSTM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fake News Classifier Using LSTM</a></span><ul class="toc-item"><li><span><a href="#importing-tensorflow-librarys" data-toc-modified-id="importing-tensorflow-librarys-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>importing tensorflow librarys</a></span></li><li><span><a href="#Onehot-Representation" data-toc-modified-id="Onehot-Representation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Onehot Representation</a></span></li><li><span><a href="#Embedding-Representation" data-toc-modified-id="Embedding-Representation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Embedding Representation</a></span></li><li><span><a href="#Model-Training" data-toc-modified-id="Model-Training-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Model Training</a></span></li><li><span><a href="#Adding-Dropout" data-toc-modified-id="Adding-Dropout-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Adding Dropout</a></span></li><li><span><a href="#Performance-Metrics-And-Accuracy" data-toc-modified-id="Performance-Metrics-And-Accuracy-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Performance Metrics And Accuracy</a></span><ul class="toc-item"><li><span><a href="#with-initial-model" data-toc-modified-id="with-initial-model-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>with initial model</a></span></li><li><span><a href="#model-with-dropout" data-toc-modified-id="model-with-dropout-2.6.2"><span class="toc-item-num">2.6.2&nbsp;&nbsp;</span>model with dropout</a></span></li></ul></li></ul></li><li><span><a href="#Fake-News-Classifier-Using-Bidirectional-LSTM" data-toc-modified-id="Fake-News-Classifier-Using-Bidirectional-LSTM-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fake News Classifier Using Bidirectional LSTM</a></span></li></ul></div>

# Importing Librarys

In [1]:
import pandas as pd
import numpy as np

# Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
df=pd.read_csv('Datafiles/fake_news_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Droping Nan Values
df = df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
# Getting Independent Features and Dependent features
X = df.drop('label',axis=1)
y = df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

## importing tensorflow librarys

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## Onehot Representation

In [15]:
# Vocabulary size
voc_size = 5000

In [16]:
messages = X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [38]:
messages.reset_index(drop=['index'], inplace=True)

In [39]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHEHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
# Dataset Preprocessing
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [43]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [44]:
onehot_repr = [one_hot(words, voc_size) for words in corpus] 
onehot_repr

[[2944, 3298, 4081, 451, 3686, 3948, 1698, 1200, 2151, 407],
 [10, 4599, 4412, 368, 2954, 113, 3174],
 [909, 1018, 4234, 295],
 [3118, 4176, 4129, 207, 3307, 3623],
 [3841, 2954, 367, 1068, 2296, 141, 2954, 4288, 1022, 4179],
 [62,
  2415,
  2706,
  1305,
  2368,
  1801,
  3028,
  4806,
  3661,
  4083,
  1419,
  1834,
  778,
  835,
  3174],
 [3559, 3320, 909, 1898, 4435, 638, 141, 345, 2550, 380, 2623],
 [2869, 3606, 3020, 1337, 4050, 1726, 1801, 4421, 2550, 380, 2623],
 [3764, 2104, 1393, 528, 4652, 4354, 2046, 373, 1801, 3936],
 [780, 2661, 545, 3089, 2204, 3918, 1019, 3985],
 [2086, 1001, 1176, 166, 3272, 4093, 3556, 2481, 3348, 624, 4015],
 [207, 1772, 3686, 4354, 1801, 4050],
 [1526, 367, 3058, 2759, 2369, 1759, 4362, 1854, 2050],
 [1476, 917, 1804, 2109, 2351, 4548, 3626, 2550, 380, 2623],
 [1801, 877, 4684, 448, 702, 2550, 380, 2623],
 [3587, 98, 4784, 176, 3564, 357, 4140, 2222, 465, 494],
 [4268, 2139, 4599],
 [2151, 3028, 3909, 148, 1801, 1117, 85, 3174],
 [2082, 2146, 4412, 

In [45]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [46]:
onehot_repr[1]

[10, 4599, 4412, 368, 2954, 113, 3174]

## Embedding Representation

In [47]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
print(embedded_docs)

[[2944 3298 4081 ...    0    0    0]
 [  10 4599 4412 ...    0    0    0]
 [ 909 1018 4234 ...    0    0    0]
 ...
 [ 830 2821 4578 ...    0    0    0]
 [4648 4050 3817 ...    0    0    0]
 [3676 1184   97 ...    0    0    0]]


In [74]:
embedded_docs[0]

array([2944, 3298, 4081,  451, 3686, 3948, 1698, 1200, 2151,  407,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [75]:
len(embedded_docs), y.shape

(18285, (18285,))

In [76]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [77]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [79]:
# Creating model
# using loss='binary_crossentropy' as it is a binary classification problem
embedding_vector_features = 40 # features representation
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 16ms/step - loss: 0.3040 - accuracy: 0.8523 - val_loss: 0.2214 - val_accuracy: 0.9117
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1430 - accuracy: 0.9463 - val_loss: 0.2008 - val_accuracy: 0.9226
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0905 - accuracy: 0.9676 - val_loss: 0.2250 - val_accuracy: 0.9205
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0617 - accuracy: 0.9799 - val_loss: 0.2198 - val_accuracy: 0.9170
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0442 - accuracy: 0.9861 - val_loss: 0.3084 - val_accuracy: 0.9168
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0314 - accuracy: 0.9905 - val_loss: 0.3290 - val_accuracy: 0.9153
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0249 - accuracy: 0.9915 - val_loss: 0.4054 - val_accuracy: 0.9102

## Adding Dropout 

In [81]:
from tensorflow.keras.layers import Dropout
# Creating model
embedding_vector_features = 40
model1=Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 15ms/step - loss: 0.3347 - accuracy: 0.8287 - val_loss: 0.1961 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1497 - accuracy: 0.9438 - val_loss: 0.1952 - val_accuracy: 0.9211
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1077 - accuracy: 0.9609 - val_loss: 0.1974 - val_accuracy: 0.9215
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0794 - accuracy: 0.9723 - val_loss: 0.2534 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0684 - accuracy: 0.9760 - val_loss: 0.2600 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0501 - accuracy: 0.9843 - val_loss: 0.2786 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0414 - accuracy: 0.9869 - val_loss: 0.3005 - val_accuracy: 0.9173

## Performance Metrics And Accuracy

### with initial model

In [83]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [84]:
y_pred = np.where(y_pred > 0.6, 1,0) #AUC ROC Curve

In [85]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3152,  267],
       [ 314, 2302]], dtype=int64)

In [86]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.903728251864126

In [87]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



### model with dropout

In [88]:
y_pred1 = model1.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [89]:
y_pred1 = np.where(y_pred1 > 0.6, 1,0) #AUC ROC Curve
confusion_matrix(y_test, y_pred1)

array([[3139,  280],
       [ 245, 2371]], dtype=int64)

In [90]:
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



# Fake News Classifier Using Bidirectional LSTM

In [91]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [92]:
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [93]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 8s 15ms/step - loss: 0.2591 - accuracy: 0.8813 - val_loss: 0.1915 - val_accuracy: 0.9193
Epoch 2/10
383/383 [==============================] - 5s 13ms/step - loss: 0.1256 - accuracy: 0.9534 - val_loss: 0.1972 - val_accuracy: 0.9231
Epoch 3/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0818 - accuracy: 0.9718 - val_loss: 0.2410 - val_accuracy: 0.9157
Epoch 4/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0557 - accuracy: 0.9812 - val_loss: 0.2736 - val_accuracy: 0.9162
Epoch 5/10
383/383 [==============================] - 5s 14ms/step - loss: 0.0412 - accuracy: 0.9869 - val_loss: 0.3535 - val_accuracy: 0.9128
Epoch 6/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0250 - accuracy: 0.9927 - val_loss: 0.4063 - val_accuracy: 0.9132
Epoch 7/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 0.5045 - val_accuracy: 0.9114

In [94]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [95]:
y_pred = np.where(y_pred>=0.5,1,0)

In [96]:
confusion_matrix(y_test,y_pred)

array([[3132,  287],
       [ 265, 2351]], dtype=int64)

In [97]:
print(accuracy_score(y_test,y_pred))

0.9085335542667772
